In [1]:
import pandas as pd
import xlrd
import time
import datetime as dt
from pandas_datareader import data
from datetime import date, timedelta
import numpy as np
import csv

In [2]:
# premumUY2017=pd.read_excel("Premium UY2017.xlsx").drop(["Policy_Holder_Name", "UY", "Effective_Date","Policy_Status",
#                                                         "Individual_NBR", "nationality", "DOB","Gender","Class","Relation"],axis=1)
# premumUY2017

- [X] merging two to create one dataset
- [X] sorted claims PPPY
- [X] call age from production to claims
- [X] sorted  premium PPPY
- [X] calculate earned premium 
- [X] assign ageband
- [X] export final result into xlsx
- [X] calculation of loss ratio and burning cost on UY
- [X] calculation of burning cost on class wise
- [X] performance display on different format such as gender & relation and so on 
- [ ] not done task

# Main Files

In [3]:
file1      = pd.read_excel("CLAIMS RAW DATA_2017_PART1.xlsx")
file2      = pd.read_excel("CLAIMS RAW DATA_2017_PART2.xlsx")

#Merge 2 claims files into one file
claimsUY17 = file1.merge(file2, how = "outer")
claimsUY17 .rename(columns={"Individual_Nbr":"Individual_NBR"},inplace=True)
claimsUY17 .head(1)
output=None

In [4]:
premiumUY17    = pd.read_excel("Premium UY2017.xlsx")
df_premiumUY17 = pd.DataFrame(premiumUY17)
df_premiumUY17.head(1)
output=None

#### Claims Data

In [5]:
# This step is to extract all other information from claims data except claims amount
df_claimsUY17        = pd.DataFrame(claimsUY17)
claimsUY17_OTH_Trim  = df_claimsUY17[["Policy_Type","Policy_Holder_Name","Effective_Date","Expiry_date","Individual_NBR",
                                      "relationship","nationality","class","Date_Of_Birth","Gender","Provider_Name",
                                      "Transaction_date","Claim_Status","Sub_Category","Disease_Description"]]
claimsUY17_OTH_Field = claimsUY17_OTH_Trim.drop_duplicates("Individual_NBR")
claimsUY17_OTH_Field .head(1)

,Policy_Type,Policy_Holder_Name,Effective_Date,Expiry_date,Individual_NBR,relationship,nationality,class,Date_Of_Birth,Gender,Provider_Name,Transaction_date,Claim_Status,Sub_Category,Disease_Description
0,INDIVIDUAL,MOAEF AYEDH ALFAHMI,2017-01-09,2018-01-08,27325265,Employee,YEMENI,C,1976-01-01,M,Al Kholoud Polyclinic - Makkah,2017-05-07,Audited,Prescription Medicine,Acute anal fissure


In [6]:
# This step to accmulate claims through unique ID
claimsUY17_AMT = df_claimsUY17
claimsUY17_AMT = claimsUY17_AMT.pivot_table(values=["losses"],index=["Individual_NBR"],aggfunc="sum")
claimsUY17_AMT.head(1)

,losses
Individual_NBR,
1000515,134.67


In [7]:
# This step is to Vlookup the age from premium data to claims
CL        = df_claimsUY17[["Individual_NBR"]]
CL        = CL.drop_duplicates("Individual_NBR")
PR        = df_premiumUY17[["Individual_NBR","Age"]]
PR        = PR.drop_duplicates("Individual_NBR")
ageUY2017 = pd.merge(PR,CL,on="Individual_NBR",how="inner")
ageUY2017.head(1)

,Individual_NBR,Age
0,27910430,58


In [8]:
# Merge both claims DataFrame
claimsUY2017 = pd.merge(claimsUY17_OTH_Field,claimsUY17_AMT,on="Individual_NBR",how="inner")
claimsUY2017 = pd.merge(claimsUY2017,ageUY2017,on="Individual_NBR",how="inner")
claimsUY2017.head(1)

,Policy_Type,Policy_Holder_Name,Effective_Date,Expiry_date,Individual_NBR,relationship,nationality,class,Date_Of_Birth,Gender,Provider_Name,Transaction_date,Claim_Status,Sub_Category,Disease_Description,losses,Age
0,INDIVIDUAL,MOAEF AYEDH ALFAHMI,2017-01-09,2018-01-08,27325265,Employee,YEMENI,C,1976-01-01,M,Al Kholoud Polyclinic - Makkah,2017-05-07,Audited,Prescription Medicine,Acute anal fissure,94.51,41


#### Premium Data

In [9]:
# This step is to extract all other information from prmeium data except premium amount
premiumUY17_OTH_Trim  = df_premiumUY17        

premiumUY17_OTH_Trim  = premiumUY17_OTH_Trim[["UY","Policy_Status","Policy_Holder_Name","Effective_Date","Expiry_date",
                                          "Endorsement_date","Individual_NBR","Age","nationality", "DOB",
                                          "Gender","Class","Relation"]]

premiumUY17_OTH_Field = premiumUY17_OTH_Trim.drop_duplicates("Individual_NBR")
premiumUY17_OTH_Field .head(1)

,UY,Policy_Status,Policy_Holder_Name,Effective_Date,Expiry_date,Endorsement_date,Individual_NBR,Age,nationality,DOB,Gender,Class,Relation
0,2017,Group,SAUDI SERVICES ELECT MECHANIC,2017-07-04,2018-07-03,2017-07-04,27910430,58,SAUDI,1959-12-31,F,VIP,Principal


In [10]:
# This step to accmulate premiums through unique ID
premiumUY17_AMT = df_premiumUY17
premiumUY17_AMT = premiumUY17_AMT.pivot_table(values = ["Net_Risk_Premium","Gross_Premium"],
                                              index  = ["Individual_NBR"],aggfunc="sum")
premiumUY17_AMT .head(1)

,Gross_Premium,Net_Risk_Premium
Individual_NBR,,
1000408,975.0,680.43


In [11]:
# Merge both premium DataFrame
premiumUY2017 = pd.merge(premiumUY17_OTH_Field,premiumUY17_AMT,on="Individual_NBR",how="inner")
premiumUY2017 .head(1)

,UY,Policy_Status,Policy_Holder_Name,Effective_Date,Expiry_date,Endorsement_date,Individual_NBR,Age,nationality,DOB,Gender,Class,Relation,Gross_Premium,Net_Risk_Premium
0,2017,Group,SAUDI SERVICES ELECT MECHANIC,2017-07-04,2018-07-03,2017-07-04,27910430,58,SAUDI,1959-12-31,F,VIP,Principal,3000.0,2253.67


In [13]:
# <<<<<< FOR incomplete year or as at date >>>>>> 
# premiumUY2017["As_at_date"] = "2019-10-01"
# premiumUY2017["As_at_date"] = pd.to_datetime(premiumUY2017.As_at_date)
# premiumUY2017["EarnedDay"] = (premiumUY2017["As_at_date"] - premiumUY2017["Effective_Date"]).dt.days.astype('int64')

# This step is required to add 1 days (premiumUY17_AMT["Expiry_date"] + pd.DateOffset(days=1)
# covert into days however due to datetime we have to convert into integer
premiumUY2017["EarnedDays"]           = ((premiumUY2017["Expiry_date"] + pd.DateOffset(days=1)) - premiumUY2017
                                         ['Effective_Date']).dt.days.astype('int64')

# Calculating the earned premium * for incomplete use as at date

premiumUY2017["Net_Earned_Premium"]    = premiumUY2017["Net_Risk_Premium"] * (premiumUY2017["EarnedDays"]/365)

# drop the heading because we dont required 
premiumUY2017.drop(["EarnedDays"],axis = 1,inplace = True)
premiumUY2017.head(1)

,UY,Policy_Status,Policy_Holder_Name,Effective_Date,Expiry_date,Endorsement_date,Individual_NBR,Age,nationality,DOB,Gender,Class,Relation,Gross_Premium,Net_Risk_Premium,Net_Earned_Premium
0,2017,Group,SAUDI SERVICES ELECT MECHANIC,2017-07-04,2018-07-03,2017-07-04,27910430,58,SAUDI,1959-12-31,F,VIP,Principal,3000.0,2253.67,2253.67


In [14]:
# Concat the claims and final dataFrame of Prmeium 
UY2017 = pd.merge(premiumUY2017,claimsUY17_AMT,on="Individual_NBR",how= "outer" )
# fill the ZERO value where claims amounts are not avaiable 
UY2017 ["losses"].fillna(0,inplace=True)
UY2017.head(1)

,UY,Policy_Status,Policy_Holder_Name,Effective_Date,Expiry_date,Endorsement_date,Individual_NBR,Age,nationality,DOB,Gender,Class,Relation,Gross_Premium,Net_Risk_Premium,Net_Earned_Premium,losses
0,2017,Group,SAUDI SERVICES ELECT MECHANIC,2017-07-04,2018-07-03,2017-07-04,27910430,58,SAUDI,1959-12-31,F,VIP,Principal,3000.0,2253.67,2253.67,21218.47


In [15]:
# Bin means set the ageBand distribution
bins              = [0,10,20,30,40,50,60,65,np.inf]
# Label the ageBand braket
AgeBracket        = ['0-10','10-20','20-30','30-40','40-50','50-60','60-65', '65+']
UY2017["AgeBand"] = pd.cut(UY2017["Age"], bins, labels=AgeBracket,include_lowest=True)
UY2017_w_AgeBand  = UY2017
UY2017_w_AgeBand  .head(1)

,UY,Policy_Status,Policy_Holder_Name,Effective_Date,Expiry_date,Endorsement_date,Individual_NBR,Age,nationality,DOB,Gender,Class,Relation,Gross_Premium,Net_Risk_Premium,Net_Earned_Premium,losses,AgeBand
0,2017,Group,SAUDI SERVICES ELECT MECHANIC,2017-07-04,2018-07-03,2017-07-04,27910430,58,SAUDI,1959-12-31,F,VIP,Principal,3000.0,2253.67,2253.67,21218.47,50-60


#### Export the DataFrame into Excel

In [17]:
UY2017 = pd.ExcelWriter("Final_Result_UY2017.xlsx")

In [18]:
UY2017_w_AgeBand.to_excel(UY2017,sheet_name="Summary",index=False)
claimsUY2017.to_excel(UY2017,sheet_name="claims",index=False)
UY2017.save()

In [19]:
# claimsUY2017.to_csv("claims_2017",index = False)

#### Analysis

In [20]:
# This pivot is the show summary of each underwriting year
# margin = Ture means total will appear and margin name means whatever you want name it 
pivot_result = UY2017_w_AgeBand.pivot_table(values=["Individual_NBR","Net_Risk_Premium","Net_Earned_Premium",
                                                    "Gross_Premium","losses"],
                  index   =    ["UY","Policy_Status"],
                  columns =    [],
                  aggfunc =    {"Individual_NBR" : len ,"Net_Risk_Premium":np.sum,"Net_Earned_Premium":np.sum,
                                "Gross_Premium":np.sum, "losses":np.sum},fill_value=(0),margins=True, 
                                            margins_name='Grand_Total').astype("int")

# This step to use to sequence the result heading
UY_Summary = pivot_result       [["Individual_NBR","Net_Risk_Premium","Net_Earned_Premium","Gross_Premium","losses"]]
UY_Summary["Burning_Cost"]   =  (UY_Summary["losses"] / UY_Summary["Individual_NBR"]).round(2)
UY_Summary["Net_Earned_LR"]  =  (UY_Summary["losses"] / UY_Summary["Net_Earned_Premium"]* 100).round(2).astype(str) + '%'

# below step to add comma in the result
UY_Summary['Individual_NBR']     = UY_Summary.apply(lambda x: "{:,}".format(x['Individual_NBR']), axis=1)
UY_Summary['Net_Risk_Premium']   = UY_Summary.apply(lambda x: "{:,}".format(x['Net_Risk_Premium']), axis=1)
UY_Summary['Net_Earned_Premium'] = UY_Summary.apply(lambda x: "{:,}".format(x['Net_Earned_Premium']), axis=1)
UY_Summary['Gross_Premium']      = UY_Summary.apply(lambda x: "{:,}".format(x['Gross_Premium']), axis=1)
UY_Summary['losses']             = UY_Summary.apply(lambda x: "{:,}".format(x['losses']), axis=1)
UY_Summary['Burning_Cost']       = UY_Summary.apply(lambda x: "{:,}".format(x['Burning_Cost']), axis=1)
UY_Summary

C:\Anaconda3\anaconda3\envs\myproject\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Anaconda3\anaconda3\envs\myproject\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Anaconda3\anaconda3\envs\myproject\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

Individual_NBR Net_Risk_Premium Net_Earned_Premium  \
UY          Policy_Status                                                      
2017        Group                 57,609       38,993,416         38,993,416   
            Individual           194,337       54,636,744         54,636,744   
Grand_Total                      251,946       93,630,161         93,630,161   

                          Gross_Premium       losses Burning_Cost  \
UY          Policy_Status                                           
2017        Group            51,829,446   52,416,631       909.87   
            Individual       79,302,758   57,110,198       293.87   
Grand_Total                 131,132,205  109,526,830       434.72   

                          Net_Earned_LR  
UY          Policy_Status                
2017        Group               134.42%  
            Individual          104.53%  
Grand_Total                     116.98%

In [21]:
# This pivot is the show summary of each underwriting year
# margin = Ture means total will appear and margin name means whatever you want name it 
pivot_result = UY2017_w_AgeBand.pivot_table(values=["Individual_NBR","Net_Risk_Premium","Net_Earned_Premium",
                                                    "Gross_Premium","losses"],
                  index   =    ["UY","Policy_Status","Class"],
                  columns =    [],
                  aggfunc =    {"Individual_NBR" : "count" ,"Net_Risk_Premium":"sum","Net_Earned_Premium":"sum",
                                "Gross_Premium":"sum", "losses":"sum"},fill_value=(0),margins=True, 
                                            margins_name='Grand_Total').astype("int")

# This step to use to sequence the result heading
Class_Wise=pivot_result       [["Individual_NBR","Net_Risk_Premium","Net_Earned_Premium","Gross_Premium","losses"]]
Class_Wise["Burning_Cost"]    = (Class_Wise["losses"] / Class_Wise["Individual_NBR"]).round(2)
Class_Wise["Net_Earned_LR"]   = (Class_Wise["losses"] / Class_Wise["Net_Earned_Premium"]* 100).round(2).astype(str) + '%'

# below step to add comma in the result
Class_Wise['Individual_NBR']     = Class_Wise.apply(lambda x: "{:,}".format(x['Individual_NBR']), axis=1)
Class_Wise['Net_Risk_Premium']   = Class_Wise.apply(lambda x: "{:,}".format(x['Net_Risk_Premium']), axis=1)
Class_Wise['Net_Earned_Premium'] = Class_Wise.apply(lambda x: "{:,}".format(x['Net_Earned_Premium']), axis=1)
Class_Wise['Gross_Premium']      = Class_Wise.apply(lambda x: "{:,}".format(x['Gross_Premium']), axis=1)
Class_Wise['losses']             = Class_Wise.apply(lambda x: "{:,}".format(x['losses']), axis=1)
Class_Wise['Burning_Cost']       = Class_Wise.apply(lambda x: "{:,}".format(x['Burning_Cost']), axis=1)
Class_Wise

C:\Anaconda3\anaconda3\envs\myproject\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Anaconda3\anaconda3\envs\myproject\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Anaconda3\anaconda3\envs\myproject\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

Individual_NBR Net_Risk_Premium  \
UY          Policy_Status Class                                   
2017        Group         A              3,769        7,970,201   
                          B              8,588        9,956,226   
                          C             44,439       17,652,470   
                          VIP              813        3,414,517   
            Individual    A                  5           14,018   
                          C            194,332       54,622,726   
Grand_Total                            251,946       93,630,161   

                                Net_Earned_Premium Gross_Premium       losses  \
UY          Policy_Status Class                                                 
2017        Group         A              7,970,201    10,386,501   12,123,693   
                          B              9,956,226    13,293,584   12,837,391   
                          C             17,652,470    23,635,844   21,744,126   
                          VIP            3,414,517     4,513,516    5,711,420   
            Individual    A                 14,018        19,191        7,146   
                          C             54,622,726    79,283,566   57,103,052   
Grand_Total                             93,630,161   131,132,205  109,526,830   

                                Burning_Cost Net_Earned_LR  
UY          Policy_Status Class                             
2017        Group         A         3,216.69       152.11%  
                          B         1,494.81       128.94%  
                          C            489.3       123.18%  
                          VIP       7,025.12       167.27%  
            Individual    A          1,429.2        50.98%  
                          C           293.84       104.54%  
Grand_Total                           434.72       116.98%

In [22]:
pivot_result = UY2017_w_AgeBand.pivot_table(values = ["Individual_NBR","losses"],
                  columns = ["Class"],
                  index   = ["UY","Policy_Status","AgeBand"], 
                  aggfunc = {"Individual_NBR" : len, "losses":np.sum},fill_value=(0)).round(2)
classWise=pivot_result      [["Individual_NBR","losses"]]

classWise_burningCost = (classWise["losses"] / classWise["Individual_NBR"]).fillna(0).round(2)
classWise_burningCost

Class                              A        B        C       VIP
UY   Policy_Status AgeBand                                      
2017 Group         0-10      1662.97   782.46   583.28   3906.23
                   10-20     1434.03   719.90   405.40   3407.11
                   20-30     2218.88  1258.13   291.34   7144.96
                   30-40     3684.13  1954.72   450.27   7024.72
                   40-50     6691.53  2065.72   626.44   6681.82
                   50-60     6830.71  3122.04  1457.19  13994.23
                   60-65    14084.55  5103.91  2143.86  16902.17
                   65+      25571.49  3275.20  2598.63  15569.17
     Individual    0-10      1520.55     0.00   226.43      0.00
                   10-20        0.00     0.00   194.61      0.00
                   20-30     5051.25     0.00   203.42      0.00
                   30-40      191.55     0.00   245.91      0.00
                   40-50        0.00     0.00   353.91      0.00
                   50-60        0.00     0.00   659.17      0.00
                   60-65        0.00     0.00   782.36      0.00
                   65+          0.00     0.00  1190.85      0.00

In [23]:
Gender_Wise = UY2017_w_AgeBand.pivot_table(values= ["Individual_NBR","Net_Risk_Premium","Net_Earned_Premium",
                                                    "Gross_Premium","losses"],
                                           index = ["UY","Policy_Status","Gender"],
                                           aggfunc= {"Individual_NBR" : "count" ,"Net_Risk_Premium":"sum",
                                                     "Net_Earned_Premium":"sum","Gross_Premium":"sum", 
                                                     "losses":"sum"},fill_value=(0),margins=True,
                                           margins_name='Grand_Total').astype("int")

Gender_Wise["Burning_Cost"]  = (Gender_Wise["losses"] / Gender_Wise["Individual_NBR"]).round(2)
Gender_Wise["Net_Earned_LR"] = (Gender_Wise["losses"] / Gender_Wise["Net_Earned_Premium"]* 100).round(2).astype(str) + '%'

# below step to add comma in the result
Gender_Wise['Individual_NBR']     = Gender_Wise.apply(lambda x: "{:,}".format(x['Individual_NBR']), axis=1)
Gender_Wise['Net_Risk_Premium']   = Gender_Wise.apply(lambda x: "{:,}".format(x['Net_Risk_Premium']), axis=1)
Gender_Wise['Net_Earned_Premium'] = Gender_Wise.apply(lambda x: "{:,}".format(x['Net_Earned_Premium']), axis=1)
Gender_Wise['Gross_Premium']      = Gender_Wise.apply(lambda x: "{:,}".format(x['Gross_Premium']), axis=1)
Gender_Wise['losses']             = Gender_Wise.apply(lambda x: "{:,}".format(x['losses']), axis=1)
Gender_Wise['Burning_Cost']       = Gender_Wise.apply(lambda x: "{:,}".format(x['Burning_Cost']), axis=1)
Gender_Wise

Gross_Premium Individual_NBR  \
UY          Policy_Status Gender                                
2017        Group         F         20,110,034         10,272   
                          M         31,719,412         47,337   
            Individual    F         24,518,614         47,929   
                          M         54,784,143        146,408   
Grand_Total                        131,132,205        251,946   

                                 Net_Earned_Premium Net_Risk_Premium  \
UY          Policy_Status Gender                                       
2017        Group         F              15,132,432       15,132,432   
                          M              23,860,983       23,860,983   
            Individual    F              16,797,180       16,797,180   
                          M              37,839,564       37,839,564   
Grand_Total                              93,630,161       93,630,161   

                                       losses Burning_Cost Net_Earned_LR  
UY          Policy_Status Gender                                          
2017        Group         F        18,815,649     1,831.74       124.34%  
                          M        33,600,982       709.82       140.82%  
            Individual    F        19,018,712       396.81       113.23%  
                          M        38,091,486       260.17       100.67%  
Grand_Total                       109,526,830       434.72       116.98%

In [24]:
Class    = UY2017_w_AgeBand.groupby(["Relation","Gender","nationality","AgeBand","Class"])
Relations = UY2017_w_AgeBand.groupby(["Policy_Status","Class","Relation"])
Genders   = UY2017_w_AgeBand.groupby(["Gender"]) 

UY2017_w_AgeBand.head(1)

,UY,Policy_Status,Policy_Holder_Name,Effective_Date,Expiry_date,Endorsement_date,Individual_NBR,Age,nationality,DOB,Gender,Class,Relation,Gross_Premium,Net_Risk_Premium,Net_Earned_Premium,losses,AgeBand
0,2017,Group,SAUDI SERVICES ELECT MECHANIC,2017-07-04,2018-07-03,2017-07-04,27910430,58,SAUDI,1959-12-31,F,VIP,Principal,3000.0,2253.67,2253.67,21218.47,50-60


In [25]:
# df=UY2017_w_AgeBand.groupby(["Relation","Class"]).Net_Risk_Premium.sum().round(2)
# df.plot(kind='bar')
df=UY2017_w_AgeBand.groupby(["Relation","Class"]).Net_Risk_Premium.sum().round(2)
df
Genders.get_group("F")["Net_Risk_Premium"].sum().round(2)
# Relation.get_group("Principal")["Net_Risk_Premium"].sum().round(2)

31929612.97

In [37]:
df=Relations.agg({"Individual_NBR":len,"Net_Risk_Premium":np.sum,"Net_Earned_Premium":np.sum,"losses":np.sum}).round(2)
df["Burning_Cost"] = df["losses"]/df["Individual_NBR"].round(2)
df["Net_Earned_LR"] = (df["losses"]/df["Net_Earned_Premium"]*100).round(2).astype(str) + '%'
df

Individual_NBR  Net_Risk_Premium  \
Policy_Status Class Relation                                      
Group         A     Child                1706        2756568.47   
                    Principal            1309        2475219.97   
                    Spouse                754        2738413.02   
              B     Child                2210        2145761.65   
                    Principal            5238        5296041.54   
                    Spouse               1140        2514423.79   
              C     Child                3443        1559179.73   
                    Other                   3           6446.21   
                    Principal           39707       13973769.44   
                    Spouse               1286        2113074.90   
              VIP   Child                 394        1199819.26   
                    Principal             265        1322846.83   
                    Spouse                154         891851.78   
Individual    A     Child                   1           2922.96   
                    Principal               3           8172.94   
                    Spouse                  1           2922.96   
              C     Child               55628        7815275.75   
                    Other                 669         385848.08   
                    Principal          119433       34777472.85   
                    Spouse              18602       11644129.37   

                               Net_Earned_Premium       losses  Burning_Cost  \
Policy_Status Class Relation                                                   
Group         A     Child              2756568.47   2601672.20   1525.013013   
                    Principal          2475219.97   5570901.73   4255.845477   
                    Spouse             2738413.02   3951119.59   5240.211658   
              B     Child              2145761.65   1882633.28    851.870262   
                    Principal          5296041.54   7549874.90   1441.365960   
                    Spouse             2514423.79   3404883.29   2986.739728   
              C     Child              1559179.73   1788901.94    519.576515   
                    Other                 6446.21     32320.60  10773.533333   
                    Principal         13973769.44  17764117.41    447.379994   
                    Spouse             2113074.90   2158786.37   1678.683025   
              VIP   Child              1199819.26   1515850.84   3847.337157   
                    Principal          1322846.83   2800652.71  10568.500792   
                    Spouse              891851.78   1394916.81   9057.901364   
Individual    A     Child                 2922.96      1520.55   1520.550000   
                    Principal             8172.94       574.64    191.546667   
                    Spouse                2922.96      5051.25   5051.250000   
              C     Child              7815275.75  11748043.17    211.189386   
                    Other               385848.08    934039.18   1396.172167   
                    Principal         34777472.85  32140137.19    269.106002   
                    Spouse            11644129.37  12280832.51    660.188824   

                              Net_Earned_LR  
Policy_Status Class Relation                 
Group         A     Child            94.38%  
                    Principal       225.07%  
                    Spouse          144.29%  
              B     Child            87.74%  
                    Principal       142.56%  
                    Spouse          135.41%  
              C     Child           114.73%  
                    Other           501.39%  
                    Principal       127.12%  
                    Spouse          102.16%  
              VIP   Child           126.34%  
                    Principal       211.71%  
                    Spouse          156.41%  
Individual    A     Child            52.02%  
                    Principal         7.03%  
             

In [38]:
df1 = Genders.agg({"Individual_NBR":len,"Net_Risk_Premium":np.sum,"Net_Earned_Premium":np.sum,"losses":np.sum}).round(2)
df1["Burning_Cost"] = df1["losses"]/df1["Individual_NBR"].round(2)
df1["Net_Earned_LR"] = (df1["losses"]/df1["Net_Earned_Premium"]*100).round(2).astype(str) + '%'
df1

,Individual_NBR,Net_Risk_Premium,Net_Earned_Premium,losses,Burning_Cost,Net_Earned_LR
Gender,,,,,,
F,58201,31929612.97,31929612.97,37834361.61,650.063772,118.49%
M,193745,61700548.53,61700548.53,71692468.55,370.035193,116.19%


In [28]:
df = pd.DataFrame(columns= UY2017_w_AgeBand.columns)
df

,UY,Policy_Status,Policy_Holder_Name,Effective_Date,Expiry_date,Endorsement_date,Individual_NBR,Age,nationality,DOB,Gender,Class,Relation,Gross_Premium,Net_Risk_Premium,Net_Earned_Premium,losses,AgeBand


In [ ]:
for Gender, data in Genders:
    add_gender= data.nlargest(6, "losses")
    df =df.append(add_gender)
df

In [ ]:
Genders   = UY2017_w_AgeBand.groupby("Gender",group_keys=True]) 